In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")
print(os.getcwd())

C:\Users\Milosz\thesis-recsys


In [2]:
import pandas as pd
import numpy as np
import functools
import operator
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import namedtuple
import pickle

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import pad
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from models import NCF, DeepFM
from features.store import FeatureStore

torch.set_printoptions(precision=2, sci_mode=False)
torch.manual_seed(0)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
with open("data/steam/data.pkl", "rb") as f:
    data = pd.read_pickle(f)

In [5]:
train_set = data['relations_datastore'].dataframe.train.values.T
supervision_set = data['relations_datastore'].dataframe.supervision.values.T
valid_set = data['relations_datastore'].dataframe.valid.values.T
item_attr = data['items_datastore'].dataframe.df
user_attr = data['users_datastore'].dataframe.df

scheme_relations = data['relations_datastore'].scheme
scheme_items = data['items_datastore'].scheme
scheme_users = data['users_datastore'].scheme

In [14]:
train_set

array([[      0,  699164,  699171, ..., 3017741, 3017981, 3017274],
       [      0,       0,       0, ...,    1230,    1230,    1230]],
      dtype=int64)

In [15]:
supervision_set

array([[1518288, 1274242, 2584398, ..., 3040549, 2142738, 1548244],
       [      0,       0,       0, ...,    1229,    1229,    1229]],
      dtype=int64)

In [13]:
np.concatenate((train_set, supervision_set), axis=1)

array([[      0,  699164,  699171, ..., 3040549, 2142738, 1548244],
       [      0,       0,       0, ...,    1229,    1229,    1229]],
      dtype=int64)

In [6]:
with open("data/steam/matrix.pkl", "rb") as f:
    matrix = pd.read_pickle(f)

In [7]:
mp_matrix = matrix['train_csr']
val_matrix = matrix['valid_csr']

In [ ]:
# scaler_user = StandardScaler()
# user_attr_preprocess = scaler_user.fit_transform(user_attr)

# scaler_item = StandardScaler()
# item_attr_preprocess = np.copy(item_attr)
# item_attr_preprocess[:, 435:] = scaler_item.fit_transform(item_attr[:, 435:])

In [8]:
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [9]:
class DeepFMDataset(Dataset):
    def __init__(self, feature_store, edge_index, user_attr, item_attr, neg_sampl):
        self.edge_index = torch.tensor(edge_index) + 1
        self.user_attr = torch.tensor(user_attr.values)
        self.item_attr = feature_store.attr2tensor(item_attr, scheme='item_feat')
        
        self.users = self.edge_index[:, 0]
        self.items = self.edge_index[:, 1]

        self.n_users = self.user_attr.shape[0]
        self.n_items = self.item_attr.shape[0]

        self.neg_sampl = neg_sampl
    
    def __len__(self):
        return self.edge_index.shape[0]
    
    def __getitem__(self, idx):
        u_id = self.users[idx].repeat(self.neg_sampl + 1)
        i_id = torch.cat([self.items[idx].unsqueeze(0), self._approx_neg_sampl()])
        
        u_attr = self.user_attr[u_id - 1]
        i_attr = self.item_attr[i_id - 1]

        x = torch.column_stack((u_id, i_id, u_attr, i_attr))
        y = torch.tensor([1] + [0] * self.neg_sampl)
        
        return x, y

    def _approx_neg_sampl(self):
        neg_i_id = torch.randint(low=0, high=self.n_items, size=(self.neg_sampl,))
        return neg_i_id

#@timer
def collate_fn(batch):
    xs, ys = [], []
    for x, y in batch:
        xs.append(x)
        ys.append(y)
    xs = torch.cat(xs)
    ys = torch.cat(ys).to(torch.float)
    return xs, ys.unsqueeze(1)

In [10]:
def train(n_epochs, print_loss=500):
    model.train()
    batch_size = train_loader.batch_size
    
    for epoch in range(n_epochs):
        running_loss = 0.
        preds, ground_truths = [], []
        for i_batch, (batch, y_true) in enumerate(tqdm(train_loader)):
            batch, y_true = batch.to(device), y_true.to(device)
            
            y_pred = model(batch)
            loss = criterion(y_pred, y_true)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            preds.append(y_pred)
            ground_truths.append(y_true)
            running_loss += loss.item()
            
            if not ((i_batch+1) % print_loss):
                pred = torch.cat(preds, dim=0).detach().sigmoid().cpu().numpy()
                ground_truth = torch.cat(ground_truths, dim=0).detach().cpu().numpy()
                last_loss = running_loss / print_loss
                
                train_roc_auc = roc_auc_score(ground_truth, pred)
                test_loss, test_roc_auc = test()
                
                preds, ground_truths = [], []
                running_loss = 0.
                
                print(f"""batch <{i_batch}>\ntrain_loss: {last_loss} - train_roc_auc: {train_roc_auc}\n
            test_loss: {test_loss} - test_roc_auc: {test_roc_auc}\n""")
        print(f"Epoch: {epoch}, Loss: {running_loss / len(train_loader):.4f}")

In [14]:
len(train_loader)

4773

In [11]:
@torch.no_grad()
def test():
    model.eval()
    batch_size = val_loader.batch_size
    
    running_loss = 0.
    preds, ground_truths = [], []

    for i_batch, (batch, y_true) in enumerate(val_loader):
        batch, y_true = batch.to(device), y_true.to(device)
        
        y_pred = model(batch)
        loss = criterion(y_pred, y_true)
        
        preds.append(y_pred)
        ground_truths.append(y_true)
        running_loss += loss.item()
        
    pred = torch.cat(preds, dim=0).sigmoid().cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    
    test_loss = running_loss / len(val_loader)
    test_score = roc_auc_score(ground_truth, pred)

    return test_loss, test_score

In [12]:
feature_store = FeatureStore(scheme_relations, scheme_items, scheme_users, emb_dims={"sparse": 4, "varlen": 4})
train_dataset = DeepFMDataset(feature_store, train_set.T, user_attr, item_attr, neg_sampl=2)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1024, collate_fn=collate_fn, drop_last=True)
val_dataset = DeepFMDataset(feature_store, valid_set.T, user_attr, item_attr, neg_sampl=2)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1024, collate_fn=collate_fn, drop_last=True)

model = DeepFM(feature_store, hidden_dim=[128, 64], device=device)
#model = NCF(feature_store, hidden_dim=[64, 32, 8])
model = model.to(device)

C:\Users\Milosz\thesis-recsys\src\features\features.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.tensor([x.iloc[self.pad_index[0]:self.pad_index[1]].values])


In [13]:
criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
optimizer = torch.optim.RMSprop(params=model.parameters(), lr=1e-4, momentum=0.9)

In [21]:
train(n_epochs=10, print_loss=500)

 10%|███████████████████████▎                                                                                                                                                                                                      | 500/4773 [02:46<23:08:26, 19.50s/it]

batch <499>
train_loss: 1.6017923901081086 - train_roc_auc: 0.5572694701013563
test_loss: 1.3073567115300075 - test_roc_auc: 0.5838225361327352



 21%|██████████████████████████████████████████████▎                                                                                                                                                                              | 1000/4773 [05:26<19:44:11, 18.83s/it]

batch <999>
train_loss: 1.1292995933294296 - train_roc_auc: 0.6186290069379805
test_loss: 1.0728814634384136 - test_roc_auc: 0.6114129514450659



 31%|█████████████████████████████████████████████████████████████████████▍                                                                                                                                                       | 1500/4773 [08:09<17:07:01, 18.83s/it]

...

Epoch: 1, Loss: 0.0360


 10%|███████████████████████▎                                                                                                                                                                                                      | 500/4773 [02:47<22:54:59, 19.31s/it]

batch <499>
train_loss: 0.6156580771207809 - train_roc_auc: 0.829132550693512
test_loss: 0.7572749440703912 - test_roc_auc: 0.771498535213915



 15%|██████████████████████████████████▌                                                                                                                                                                                              | 732/4773 [03:36<19:53,  3.38it/s]


KeyboardInterrupt: 

In [22]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(path):
    model = DeepFM(feature_store, hidden_dim=[128, 64], device=device)
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

In [23]:
save_model(model, "models/deepfm_02.pth")

In [12]:
#model = load_model("models/deepfm_02.pth")

In [24]:
model

DeepFM(
  (V): EmbeddingNet(
    (embeddings): ModuleDict(
      (user_id): Embedding(3066722, 4, padding_idx=0)
      (app_id): Embedding(1232, 4, padding_idx=0)
      (win): Embedding(2, 4, padding_idx=0)
      (mac): Embedding(3, 4, padding_idx=0)
      (linux): Embedding(3, 4, padding_idx=0)
      (steam_deck): Embedding(2, 4, padding_idx=0)
      (rating): Embedding(6, 4, padding_idx=0)
      (tags): Embedding(416, 4, padding_idx=0)
    )
  )
  (fm): FM()
  (dnn): MLP(
    (layers): Sequential(
      (0): Linear(in_features=38, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=128, out_features=64, bias=True)
      (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=64, out_features=1, bias=True)
    )
 

In [28]:
def precision_k(reco_relevance, relevance, k=10):
    v = np.asarray(relevance.sum(axis=1).flatten(), dtype=int)[0].clip(1, k)
    bool_2d = np.vstack([np.concatenate((np.ones(i), np.zeros(k - i))) for i in v]).astype(bool)
    
    prec_k = (reco_relevance.getA().sum(axis=1, where=bool_2d) / v).mean()
    return prec_k

# def mean_average_prec(reco_relevance):
#     K = reco_relevance.shape[1]
    
#     mean_ap = 0.0
#     for k in range(1, K+1):
#         mean_ap += prec_k(reco_relevance[:, :k]) # DODAC MNOŻNIK 1/0 GDY ITEM JEST RELEWANTNY!!!
#     return mean_ap / K

def recall_k(reco_relevance, relevance, k=10):
    sum_relevant = relevance.sum(axis=1)
    return (reco_relevance.sum(axis=1) / sum_relevant).mean()

def ndcg_k(reco_relevance, relevance, k=10):
    v = np.asarray(relevance.sum(axis=1).flatten(), dtype=int)[0].clip(1, k)
    ideal_relevance = np.vstack([np.concatenate((np.ones(i), np.zeros(k - i))) for i in v])
    
    discount = 1 / np.log2(np.arange(2, k+2))
    idcg = (ideal_relevance * discount).sum(axis=1)
    dcg = (reco_relevance * discount).sum(axis=1)
    ndcg = (dcg / idcg).mean()
    
    return ndcg



@torch.no_grad()
def generate_embeddings(model, data):
    pass

@torch.no_grad()
def recommend_k(user_emb, item_emb, past_interactions=None, k=10, user_batch_size=1000):
    def remove_past_interactions(prob, user_batch):
        id_x = np.repeat(np.arange(user_batch.shape[0]), np.diff(past_interactions[user_batch].indptr))
        id_y = past_interactions[user_batch].indices
        prob[id_x, id_y] = -torch.inf
        return prob
    
    recommended_batches = []
    user_batches = torch.arange(user_emb.shape[0]).split(user_batch_size)
    for user_batch in user_batches:
        prob = (user_emb[user_batch] @ item_emb.T).sigmoid()
        prob = remove_past_interactions(prob, user_batch)
        recommended_batches.append(prob.topk(k, 1)[1])
    
    recommendations = torch.cat(recommended_batches, 0)
    return recommendations

def recommendation_relevance(recommendations, ground_truth):
    """
    Computes the relevance matrix of recommended items based on ground truth data.

    This function takes a matrix of recommended items and a ground truth sparse matrix, and calculates
    binary relevance of recommended items for each user. The relevance is determined by
    comparing the recommended items with the actual items in the ground truth.

    Args:
        recommendations (numpy.ndarray): A 2D matrix of shape (n_users, k) where k is the number of 
            recommended items per user. Each row contains indices representing the recommended 
            items for a user.
        ground_truth (scipy.csr_matrix): A sparse matrix of shape (n_users, n_items). The matrix 
            contains binary values indicating whether an item is relevant (1) or not (0) for each user.

    Returns:
        numpy.matrix: A 2D matrix of shape (n_users, k) containing the relevance scores of the
        recommended items for each user.
        
    Raises:
        ValueError: If the dimensions of 'recommendations' and 'ground_truth' do not match or
            are incompatible for matrix operations.
    """
    n_users, n_items = ground_truth.shape
    k = recommendations.shape[1]
    
    if recommendations.shape[0] != n_users:
        raise ValueError("Number of users in 'recommendations' should match 'ground_truth'.")
    
    user_idx = np.repeat(np.arange(n_users), k)
    item_idx = recommendations.flatten()
    relevance = ground_truth[user_idx, item_idx].reshape((n_users, k))  # get values under arrays of indices 
                                                                        # (user_idx and item_idx) from ground truth
    relevance_mask = np.asarray((ground_truth.sum(axis=1) != 0)).ravel()
    
    return relevance, relevance_mask

def evaluate_nn(model, mp_matrix, val_matrix, k):
    x_emb = generate_embeddings(model, val_data)
    recommendations = recommend_k(x_emb['user'], x_emb['app'], past_interactions=mp_matrix, 
                                  k=10, user_batch_size=10000).cpu().numpy()
    reco_relevance, relevance_mask = recommendation_relevance(recommendations, val_matrix)
    
    prec_k = precision_k(reco_relevance[relevance_mask], val_matrix[relevance_mask], k)
    rec_k = recall_k(reco_relevance[relevance_mask], val_matrix[relevance_mask], k)
    n_k = ndcg_k(reco_relevance[relevance_mask].getA(), val_matrix[relevance_mask], k)

    return {f"precision@{k}": prec_k, f"recall@{k}": rec_k, f"ndcg@{k}": n_k}

In [29]:
@torch.no_grad()
def evaluate(model, x):
    model.eval()
    x = x.to(device)
    y = model(x)
    
    return y.sigmoid()

In [36]:
past_interactions = mp_matrix
def remove_past_interactions(prob, user_batch):
    id_x = np.repeat(np.arange(user_batch.shape[0]), np.diff(past_interactions[user_batch].indptr))
    id_y = past_interactions[user_batch].indices
    prob[id_x, id_y] = -torch.inf
    return prob

In [40]:
recommended_batches = []
user_batches = (torch.arange(3066721) + 1).split(1000)
item_batches = (torch.arange(1231) + 1).repeat(1000, 1)

In [ ]:
# for user_batch in tqdm(user_batches):
#     X = torch.cat([user_batch.repeat(1231, 1).t().reshape(-1, 1), item_tensor], dim=1)
#     prob = evaluate(model, X)
#     prob = prob.view(1000, -1)
#     prob = remove_past_interactions(prob, user_batch)
#     recommended_batches.append(prob.topk(10, 1)[1])
    
# recommendations = torch.cat(recommended_batches, 0)

In [ ]:
from reco_env import RecoEnv
from utils import import_data_for_env
import gym

In [ ]:
env = gym.make(RecoEnv.id, **import_data_for_env())

In [ ]:
vc = rec.user_id.value_counts()

In [ ]:
vc

In [ ]:
vc[vc >= 3]

In [ ]:
plt.plot(vc)